In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import json, gzip, glob
import pandas as pd
from pathlib import Path
from typing import List
from printer_anomaly_detection.dataset.domain.sensors import *
from printer_anomaly_detection.dataset.domain.printer import *
from dataclass_csv import DataclassWriter


In [3]:
FOLDER_NAME = 'hinges_12'
NAME = Path(f'../../data/{FOLDER_NAME}/')
SENSOR_TSV_FILE = next(NAME.glob('*.tsv.gz'))


# Load sensordata

In [4]:

sensor_data: List[IMUSensorData | GasSensorData] = list()
index = None
with gzip.open(SENSOR_TSV_FILE, 'rb') as f:
    for line in f:
        data = line.decode('ascii')
        data = data.split('\t')
        datetime_ = data[0]
        datetime_ = datetime.fromisoformat(datetime_)

        try:
            sensor_type = SensorType(data[1])
        except ValueError:
            continue

        if sensor_type in {SensorType.ACCELEROMETER, SensorType.MAGNETOMETER, SensorType.GYROSCOPE}:
            sensor_data.append(
                IMUSensorData(
                    sensor_type=sensor_type,
                    datetime=datetime_,
                    x = float(data[2]),
                    y = float(data[3]),
                    z = float(data[4])
                )
            )
            pass
        elif sensor_type == SensorType.GAS:
            sensor_data.append(
                GasSensorData(
                    sensor_type=sensor_type,
                    datetime=datetime_,
                    value=float(data[2])
                )
            )
        else:
            raise Exception("Invalid sensor type")

In [5]:
imu = filter(lambda d: isinstance(d, IMUSensorData), sensor_data)
gas = filter(lambda d: isinstance(d, GasSensorData), sensor_data)

# Load printer_data

In [6]:
DATA_NAME = next(NAME.glob('*_klipper.json.gz'))
DATA_NAME = DATA_NAME.name[:-len('.json.gz')]

In [7]:
from datetime import timedelta

In [8]:
index = None
with gzip.open(f'{NAME / DATA_NAME}.index.gz', 'rb') as f:
    index = f.read()
    index = index.split('\x03'.encode('ascii'))
    index = filter(lambda i: len(i) > 0, index)
    index = [json.loads(i) for i in index]

In [9]:
data = None
with gzip.open(f'{NAME / DATA_NAME}.json.gz', 'rb') as f:
    data = f.read()
    data = data.split('\x03'.encode('ascii'))
    data = filter(lambda i: len(i) > 0, data)
    data = [json.loads(i) for i in data]

In [10]:
prev = 0
for d in filter(lambda d: 'q' in d and 'eventtime' in d['params'], data):
  eventtime = d['params']['eventtime']
  assert eventtime > prev
  prev = eventtime

In [11]:
datetimestr = DATA_NAME[:-len('_klipper')]
printer_start_time: datetime = datetime.strptime(datetimestr[:-len('000Z')], '%Y-%m-%dT%H%M%S%f')
printer_time_offset = -next(filter(lambda d: 'q' in d, data))['params']['eventtime']

In [12]:
from typing import Tuple, List, Dict, Set

In [13]:
def get_field(field_name: str, data, printer_start_time, printer_time_delta) -> List[Tuple[datetime, Dict]]:
    def _f(row):
        if 'q' not in row or row['q'] != 'status':
            return False
        return field_name in row['params']['status']

    def _m(row):
        tdelta = timedelta(seconds=(row['params']['eventtime'] + printer_time_offset))
        return printer_start_time + tdelta, row['params']['status'][field_name]
    
    return list(map(_m, filter(_f, data)))

In [14]:
def get_keys(d: Dict) -> Set[str]:
  keys = d.keys()
  for k in keys:
    if isinstance(d[k], dict):
      keys |= map(lambda i: f'{k}.{i}', get_keys(d[k]))
  return keys

In [15]:
def get_all_keys(field_name: str, data, printer_start_time, printer_time_delta):
  keys = set()
  for d in get_field(field_name, data, printer_start_time, printer_time_delta):
    new_keys = get_keys(d[1])
    
    keys |= new_keys
    
  return keys

## Fields:
* heater_bed
* tmc2209 stepper_x
* tmc2209 stepper_y
* tmc2209 stepper_z
* tmc2209 extruder
* mcu
* extruder
* motion_report
* gcode_move


## Heater

In [16]:
get_all_keys('heater_bed', data, printer_start_time, printer_time_offset)

{'power', 'target', 'temperature'}

In [17]:
heater_data = get_field('heater_bed', data, printer_start_time, printer_time_offset)

In [18]:
heater_target = map(
  lambda d: TargetData(SensorType.HEATER, datetime=d[0], target=d[1]['target']),
  filter(lambda d: 'target' in d[1], heater_data)
)

heater_power = map(
  lambda d: PowerSensorData(SensorType.HEATER, datetime=d[0], power=d[1]['power']),
  filter(lambda d: 'power' in d[1], heater_data)
)

heater_temperature = map(
  lambda d: TemperatureSensorData(SensorType.HEATER, datetime=d[0], temperature=d[1]['temperature']),
  filter(lambda d: 'temperature' in d[1], heater_data)
)

## Motordriver

In [19]:
get_all_keys('tmc2209 stepper_x', data, printer_start_time, printer_time_offset)
get_all_keys('tmc2209 stepper_y', data, printer_start_time, printer_time_offset)
get_all_keys('tmc2209 stepper_z', data, printer_start_time, printer_time_offset)
get_all_keys('tmc2209 extruder', data, printer_start_time, printer_time_offset)

{'drv_status',
 'drv_status.cs_actual',
 'drv_status.ola',
 'drv_status.olb',
 'drv_status.stealth',
 'drv_status.stst'}

## MCU

In [20]:
print(get_all_keys('mcu', data, printer_start_time, printer_time_offset))


{'last_stats', 'last_stats.send_seq', 'last_stats.rttvar', 'last_stats.bytes_write', 'last_stats.upcoming_bytes', 'last_stats.srtt', 'last_stats.bytes_invalid', 'last_stats.freq', 'last_stats.bytes_retransmit', 'last_stats.ready_bytes', 'last_stats.receive_seq', 'last_stats.mcu_task_stddev', 'last_stats.mcu_awake', 'last_stats.mcu_task_avg', 'last_stats.retransmit_seq', 'last_stats.bytes_read', 'last_stats.rto'}


In [21]:
    upcoming_bytes: int
    srtt: float
    bytes_invalid: int
    send_seq: int
    retransmit_seq: int
    rttvar: float
    rto: float
    bytes_retransmit: int
    bytes_write: int
    ready_bytes: int
    freq: int
    receive_seq: int

In [22]:
mcu_data = get_field('mcu', data, printer_start_time, printer_time_offset)

In [23]:
mcu = map(
  lambda d: MCUData(
    SensorType.MCU,
    datetime=d[0],
    upcoming_bytes=d[1]['last_stats']['upcoming_bytes'],
    srtt=d[1]['last_stats']['srtt'],
    bytes_invalid=d[1]['last_stats']['bytes_invalid'],
    send_seq=d[1]['last_stats']['send_seq'],
    retransmit_seq=d[1]['last_stats']['retransmit_seq'],
    rttvar=d[1]['last_stats']['rttvar'],
    rto=d[1]['last_stats']['rto'],
    bytes_retransmit=d[1]['last_stats']['bytes_retransmit'],
    bytes_write=d[1]['last_stats']['bytes_write'],
    ready_bytes=d[1]['last_stats']['ready_bytes'],
    freq=d[1]['last_stats']['freq'],
    receive_seq=d[1]['last_stats']['receive_seq']),
  mcu_data
)

## Extruder

In [24]:
print(get_all_keys('extruder', data, printer_start_time, printer_time_offset))

{'target', 'power', 'can_extrude', 'temperature'}


In [25]:
extruder_data = get_field('extruder', data, printer_start_time, printer_time_offset)

In [26]:
extruder_target = map(
  lambda d: TargetData(SensorType.EXTRUDER, datetime=d[0], target=d[1]['target']),
  filter(lambda d: 'target' in d[1], extruder_data)
)

extruder_power = map(
  lambda d: PowerSensorData(SensorType.EXTRUDER, datetime=d[0], power=d[1]['power']),
  filter(lambda d: 'power' in d[1], extruder_data)
)

extruder_temperature = map(
  lambda d: TemperatureSensorData(SensorType.EXTRUDER, datetime=d[0], temperature=d[1]['temperature']),
  filter(lambda d: 'temperature' in d[1], extruder_data)
)

## Motion report

In [27]:
print(get_all_keys('motion_report', data, printer_start_time, printer_time_offset))

{'live_position', 'live_velocity', 'live_extruder_velocity'}


In [28]:
motion_report_data = get_field('motion_report', data, printer_start_time, printer_time_offset)

In [29]:
position_sensor = map(
  lambda d: PositionSensorData(
    SensorType.MOTION,
    datetime=d[0],
    x=d[1]['live_position'][0],
    y=d[1]['live_position'][1],
    z=d[1]['live_position'][2],
    e=d[1]['live_position'][3],
  ),
  filter(lambda d: 'live_position' in d[1], motion_report_data)
)

In [30]:
velocity_sensor = map(
  lambda d: VelocitySensorData(
    SensorType.MOTION,
    datetime=d[0],
    velocity=d[1]['live_velocity'],
  ),
  filter(lambda d: 'live_velocity' in d[1], motion_report_data)
)

In [31]:
extruder_velocity_sensor = map(
  lambda d: VelocitySensorData(
    SensorType.EXTRUDER,
    datetime=d[0],
    velocity=d[1]['live_extruder_velocity'],
  ),
  filter(lambda d: 'live_extruder_velocity' in d[1], motion_report_data)
)

# Motion Target / GCode

In [32]:
print(get_all_keys('gcode_move', data, printer_start_time, printer_time_offset))

{'position', 'speed', 'gcode_position', 'absolute_coordinates'}


In [33]:
gcode_data = get_field('gcode_move', data, printer_start_time, printer_time_offset)

In [34]:
position_target = map(
  lambda d: PositionTargetData(
    SensorType.MOTION,
    datetime=d[0],
    x=d[1]['position'][0],
    y=d[1]['position'][1],
    z=d[1]['position'][2],
    e=d[1]['position'][3],
  ),
  filter(lambda d: 'position' in d[1], gcode_data)
)

In [35]:
velocity_target = map(
  lambda d: VelocityTargetData(
    SensorType.MOTION,
    datetime=d[0],
    velocity=d[1]['speed'],
  ),
  filter(lambda d: 'speed' in d[1], gcode_data)
)

# Now save everything

In [36]:
DATASET_NAME = Path(f'../../datasets/{FOLDER_NAME}/')
DATASET_NAME.mkdir(parents=True, exist_ok=True)

In [37]:
everything = zip(['imu',
'gas',
'heater_target',
'heater_power',
'heater_temperature',
'mcu',
'extruder_target',
'extruder_power',
'extruder_temperature',
'position_sensor',
'velocity_sensor',
'extruder_velocity_sensor',
'position_target',
'velocity_target'], [imu,
gas,
heater_target,
heater_power,
heater_temperature,
mcu,
extruder_target,
extruder_power,
extruder_temperature,
position_sensor,
velocity_sensor,
extruder_velocity_sensor,
position_target,
velocity_target], [
  IMUSensorData,
  GasSensorData,
  TargetData,
  PowerSensorData,
  TemperatureSensorData,
  MCUData,
  TargetData,
  PowerSensorData,
  TemperatureSensorData,
  PositionSensorData,
  VelocitySensorData,
  VelocitySensorData,
  PositionTargetData,
  VelocityTargetData
])

In [38]:
for name, sensor, cls in everything:
  print(name, sensor)
  with gzip.open(DATASET_NAME / f'{name}.csv.gz', 'wt') as f:
    writer = DataclassWriter(f, list(sensor), cls)
    writer.write()

imu <filter object at 0x1421c39a0>


KeyboardInterrupt: 

# Video

In [39]:
from ffmpeg import FFmpeg

In [49]:
video_path = next(NAME.glob('*.mp4'))

In [52]:
ffmpeg = (
    FFmpeg()
    .option("y")
    .input(str(video_path))
    .output(
        "output.mp4",
        {"codec:a": "copy"},
        map="0:a",
    )
)
ffmpeg.execute()

b''

# Remarks

In [73]:
DATA_FOLDER = Path(f'../../data/')

In [86]:
data_paths = filter(lambda p: p.is_dir(), DATA_FOLDER.glob('*'))

for data_path in data_paths:
    remarks = filter(lambda p: p.is_file() and 'remark' in p.name.lower(), data_path.iterdir())
    failures = filter(lambda p: p.is_file() and 'fail' in p.name.lower(), data_path.iterdir())


    if failure := bool(next(failures, None)):
        #print(data_path.name)
        #print(f'failed')
        #print()
        pass

    if remarks := next(remarks, None):
        if failure:
            continue
        with open(remarks, 'r', encoding='utf-8') as f:
            print(data_path.name)
            print('remarks')
            print(f.read())
            print()

hinges_4
remarks
Anomalie am Anfang
Lüfter gestartet, nachdem Betttemp. hoch genug war


hinges_3
remarks
Fan erst nach 60 Minuten eingeschaltet


